In [2]:
import face_recognition
import numpy as np

In [3]:
!pip3 install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import cv2
print(cv2.__version__)


4.10.0


In [5]:
video_capture = cv2.VideoCapture(0)

In [6]:
import os
import csv
from datetime import datetime

In [7]:
if not os.path.exists('registered_faces'):
    os.makedirs('registered_faces')

In [8]:
# Load a sample picture and learn how to recognize it.
Modi_image = face_recognition.load_image_file("Modi.jpg")
Modi_face_encoding = face_recognition.face_encodings(Modi_image)[0]

# Load a second sample picture and learn how to recognize it.
Kohli_image = face_recognition.load_image_file("Kohli.jpg")
Kohli_face_encoding = face_recognition.face_encodings(Kohli_image)[0]

Nihar_image = face_recognition.load_image_file("Nihar.jpg")
Nihar_face_encoding = face_recognition.face_encodings(Nihar_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    Modi_face_encoding,
    Kohli_face_encoding,
    Nihar_face_encoding
]
known_face_names = [
    "Modi",
    "Kohli",
    "Nihar"
]
# Aadhaar details for the known faces
aadhaar_details = {
    "Modi": "Aadhaar Details for Modi: [1234-5678-9101]",
    "Kohli": "Aadhaar Details for Kohli: [2345-6789-1012]",
    "Nihar": "Aadhaar Details for Nihar: [3456-7890-1123]"
}

In [9]:
Kohli_face_encoding

array([-0.09965005,  0.01257287,  0.03549203, -0.05077933, -0.02275129,
       -0.05707148, -0.00739357, -0.04482555,  0.17687564, -0.0691358 ,
        0.12299565, -0.02379135, -0.20054556, -0.08880908,  0.00350207,
        0.01744138, -0.16425145, -0.15290068, -0.00792788, -0.06312364,
        0.04632288,  0.0254629 ,  0.06577305,  0.06097015, -0.21376604,
       -0.30779284, -0.08107652, -0.21990243,  0.01530453, -0.13512173,
       -0.00346159,  0.04897284, -0.20438799, -0.0503398 , -0.03567981,
        0.10015926,  0.08204275, -0.04600226,  0.20110808,  0.00590375,
       -0.14903857, -0.08944672, -0.04790143,  0.26564246,  0.05679313,
       -0.02259418,  0.02646403, -0.01758976,  0.13822114, -0.21441488,
        0.09875869,  0.13483532,  0.0633258 ,  0.00693662,  0.06015252,
       -0.1252597 ,  0.04251543,  0.08185501, -0.21970846,  0.04835903,
        0.04016885,  0.05355866,  0.02343511, -0.00812267,  0.17923084,
        0.03975492, -0.09150098, -0.05622239,  0.05830463, -0.14

# Detecting Stored faces and giving back their aadhar data

In [10]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
recognized_faces = set()  # Set to keep track of recognized faces

# Capture video from the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

            # Print Aadhaar details if the face is recognized and not already printed
            if name in aadhaar_details and name not in recognized_faces:
                print(aadhaar_details[name])
                recognized_faces.add(name)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

Aadhaar Details for Nihar: [3456-7890-1123]


KeyboardInterrupt: 

# Detecting new faces and storing their aadhar data

In [13]:
import tkinter as tk
from tkinter import simpledialog

In [9]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
recognized_faces = set()  # Set to keep track of recognized faces

# Initialize Tkinter
root = tk.Tk()
root.withdraw()  # Hide the root window

# Capture video from the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        if face_encodings:  # If at least one face encoding is found
            face_encoding = face_encodings[0]  # Process only the first detected face

            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names = [name]

            # Print Aadhaar details if the face is recognized and not already printed
            if name in aadhaar_details and name not in recognized_faces and name != "Nihar":
                print(aadhaar_details[name])
                recognized_faces.add(name)
            elif name == "Unknown":
                # Show dialog box to register new face
                response = simpledialog.askstring("Input", "Unknown face detected. Do you want to register this face? (yes/no)", parent=root)
                if response and response.lower() == "yes":
                    new_name = simpledialog.askstring("Input", "Enter name:", parent=root)
                    new_aadhaar = simpledialog.askstring("Input", "Enter Aadhaar details:", parent=root)
                    if new_name and new_aadhaar:
                        known_face_encodings.append(face_encoding)
                        known_face_names.append(new_name)
                        aadhaar_details[new_name] = f"Aadhaar Details for {new_name}: {new_aadhaar}"
                        print(f"Registered new face: {new_name}")

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if name == "Nihar":
            continue  # Skip drawing the box and label for Nihar
        
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()

Registered new face: Hardik
Aadhaar Details for Hardik: 0011-0011-0011


# Code for New data storage 

In [11]:
!pip install numpy==1.24.4


Defaulting to user installation because normal site-packages is not writeable

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.24.4 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.



  Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp38-cp38-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3


In [12]:
import numpy as np
print(np.__version__)


1.24.3


In [13]:
import pandas as pd

In [14]:

from datetime import datetime
from tkinter import simpledialog, Tk

In [15]:
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
recognized_faces = set()  # Set to keep track of recognized faces

# Initialize Tkinter
root = Tk()
root.withdraw()  # Hide the root window

# Capture video from the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        if face_encodings:  # If at least one face encoding is found
            face_encoding = face_encodings[0]  # Process only the first detected face

            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names = [name]

            # Print Aadhaar details if the face is recognized and not already printed
            if name in aadhaar_details and name not in recognized_faces and name != "Nihar":
                print(aadhaar_details[name])
                recognized_faces.add(name)
            elif name == "Unknown":
                # Show dialog box to register new face
                response = simpledialog.askstring("Input", "Unknown face detected. Do you want to register this face? (yes/no)", parent=root)
                if response and response.lower() == "yes":
                    new_name = simpledialog.askstring("Input", "Enter name:", parent=root)
                    new_aadhaar = simpledialog.askstring("Input", "Enter Aadhaar details:", parent=root)
                    if new_name and new_aadhaar:
                        known_face_encodings.append(face_encoding)
                        known_face_names.append(new_name)
                        aadhaar_details[new_name] = f"Aadhaar Details for {new_name}: {new_aadhaar}"
                        print(f"Registered new face: {new_name}")

                        # Save the new face image
                        top, right, bottom, left = face_locations[0]
                        top *= 4
                        right *= 4
                        bottom *= 4
                        left *= 4
                        face_image = frame[top:bottom, left:right]
                        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
                        face_image_path = os.path.join('registered_faces', f"{new_name}_{timestamp}.jpg")
                        cv2.imwrite(face_image_path, face_image)
                        
                        # Save the new face data to a CSV file
                        face_data = pd.DataFrame([[new_name, new_aadhaar, face_image_path]], columns=["Name", "Aadhaar", "Image_Path"])
                        csv_path = 'registered_faces_data.csv'
                        if os.path.exists(csv_path):
                            face_data.to_csv(csv_path, mode='a', header=False, index=False)
                        else:
                            face_data.to_csv(csv_path, mode='w', header=True, index=False)

    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        if name == "Nihar":
            continue  # Skip drawing the box and label for Nihar
        
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()